Testing theory power spectra generator script

In [ ]:
import sys
import pickle
import camb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0,'/global/homes/t/tanveerk/SkyLens/') #path to skylens
sys.path.insert(0,'/global/homes/t/tanveerk/SkyLens/skylens') #path to skylens
sys.path.insert(0,'/global/homes/t/tanveerk/desi-lensing-cc/src/') #path to helper functions

import healpy as hp
#from skylens import *
#from survey_utils import *
import skylens
import survey_utils
import utils #helper functions

#only for python3
import importlib
reload=importlib.reload

from time import time
from scipy.integrate import quad